# Modelo de Classificação Binária - Predição de compras de clientes

Este _notebook_ implementa o treinamento de um modelo de classificação binária para prever se, com base em um conjunto de atributos, um cliente realizará compras ou não. O conjunto de dados utilizado é o `customer-purchase-behavior.csv`, o qual pode ser acessado na pasta `data`. Ele foi obtido através da plataforma Kaggle, [neste _link_](https://www.kaggle.com/datasets/rabieelkharoua/predict-customer-purchase-behavior-dataset).

As redes foram treinadas com diferentes configurações para avaliar o desempenho dos modelos com diferentes funções de ativação, épocas e _hidden layers_. Para o treinamento dos modelos, a biblioteca de Rede Neural Artificial desenvolvida pelos membros do grupo foi utilizada. Mais precisamente, a classe `NeuralNetwork`, a qual representa a rede, foi instanciada e os modelos foram treinados através dos métodos implementados pela classe.

## Dependências

Esta célula é responsável pela importação das dependências utilizadas para os treinamentos dos modelos. Mais precisamente, foram utilizadas como dependências a classe representando a Rede Neural Artificial (`NeuralNetwork`); `pl` (Polars) e `np` (NumPy) para interação com o _dataset_; e, dependências do `scikit-learn` para normalização de dados e divisão do conjunto de dados em dados de teste e de validação.

In [5]:
from neural_lib.neural_network import NeuralNetwork

import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## Carregamento e normalização dos dados

Aqui, os dados do _dataset_ são carregados em memória e as seguintes normalizações são realizadas:
- Escalonamento das variáveis contínuas (`continuos_features`) com o `StandardScaler`
- _One-hot encoding_ para a variável categórica `ProductCategory`
- Manutenção das variáveis binárias (`Gender` e `LoyaltyProgram`), visto que elas já assumem valores `0` e `1`

In [6]:
def load_and_preprocess(csv_path):
    df = pl.read_csv(csv_path)

    continuos_features = [
        'Age', 'AnnualIncome', 'NumberOfPurchases',
        'TimeSpentOnWebsite', 'DiscountsAvailed'
    ]
    categorical_features = ['ProductCategory']
    passthrough_feats = ['Gender', 'LoyaltyProgram']

    features = continuos_features + categorical_features + passthrough_feats
    X_raw = df.select(features).to_numpy()
    y = df.select('PurchaseStatus').to_numpy().flatten()

    preprocessor = ColumnTransformer([
        ('scale', StandardScaler(), list(range(0, len(continuos_features)))),
        ('onehot', OneHotEncoder(sparse_output=False), [len(continuos_features)]),
    ], remainder='passthrough')

    X = preprocessor.fit_transform(X_raw)
    return X, y

## Definição do modelo e _helper functions_

A função `train_model` cria e treina uma instância da rede neural. A rede é instanciada com a _loss function_ `binary`, função de ativação de saída `sigmoid` e um único neurônio de saída, visto que o modelo a ser treinado trata-se de um classificador binário.

A função `evaluate_model` avalia o desempenho do modelo em função do conjunto de dados de validação. A acurácia do modelo é retornada.

In [7]:
def train_model(X_train, y_train, hidden_layers, epochs, activation_function):
    input_size = X_train.shape[1]
    neural_network = NeuralNetwork(
        input_size=input_size,
        hidden_sizes=hidden_layers,
        activation=activation_function,
        output_size=1,
        output_activation='sigmoid',
        loss='binary'
    )

    neural_network.train(X_train, y_train, epochs)
    return neural_network

def evaluate_model(neural_network, X_test, y_test):
    acc = neural_network.predict(X_test, y_test)
    return acc

## Treinamento e avaliação dos modelos

A função `get_params()` retorna um conjunto de `hidden_layers`, `activation_function` e `epochs` a serem utilizadas para avaliar o desempenho do modelo.

Em seguida, o conjunto de dados é dividido em dados de teste e de treinamento. Para cada conjunto de parâmetros definido, o modelo de classificação binária é treinado e avaliado.

In [8]:
def get_params():
    return [
        {'hidden_layers': [32, 16], 'activation_function': 'relu',    'epochs': 100},
        {'hidden_layers': [32, 16], 'activation_function': 'tanh',    'epochs': 100},

        {'hidden_layers': [64, 32, 16], 'activation_function': 'relu',    'epochs': 200},
        {'hidden_layers': [64, 32, 16], 'activation_function': 'tanh',    'epochs': 200},

        {'hidden_layers': [128, 64, 32], 'activation_function': 'relu',    'epochs': 300},
        {'hidden_layers': [128, 64, 32], 'activation_function': 'tanh',    'epochs': 300},
    ]

X, y = load_and_preprocess('./../data/customer-purchase-behavior.csv')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

results = []
for params in get_params():
    hidden_layers = params['hidden_layers']
    epochs = params['epochs']
    activation_function = params['activation_function']

    neural_network = train_model(
        X_train, y_train,
        hidden_layers,
        epochs,
        activation_function
    )
    acc = evaluate_model(neural_network, X_test, y_test)
    print(f'Hidden Layers: {hidden_layers}, Epochs: {epochs}, Activation Function: {activation_function} -> Accuracy: {acc:.2f}%')

Accuracy on last epoch 96.0
Hidden Layers: [32, 16], Epochs: 100, Activation Function: relu -> Accuracy: 46.33%
Accuracy on last epoch 77.0
Hidden Layers: [32, 16], Epochs: 100, Activation Function: tanh -> Accuracy: 59.00%
Accuracy on last epoch 4.0
Hidden Layers: [64, 32, 16], Epochs: 200, Activation Function: relu -> Accuracy: 47.33%
Accuracy on last epoch 21.5
Hidden Layers: [64, 32, 16], Epochs: 200, Activation Function: tanh -> Accuracy: 59.67%
Accuracy on last epoch 2.3333333333333335
Hidden Layers: [128, 64, 32], Epochs: 300, Activation Function: relu -> Accuracy: 46.00%
Accuracy on last epoch 22.333333333333332
Hidden Layers: [128, 64, 32], Epochs: 300, Activation Function: tanh -> Accuracy: 60.67%
